## PromptTempalte
- Langchain의 PromptTemplate은 프롬프트를 동적으로 생성할 수 있도록 도와주는 도구입니다
- 변수와 템플릿을 활용해 다양한 입력값에 맞는 프롬프트를 손쉽게 만들 수 있습니다.  
    - `input_variables` : 중괄호 안에 들어갈 변수이름 리스트

공식문서에서 더 다양한 종류의 PromptTemplate들을 확인 할 수 있습니다.
- https://python.langchain.com/api_reference/core/prompts.html

Langchain_ollama 라이브리러를 이용하여 확인해보기

### langchain_ollama
- `langchain_ollama`는 LangChain 프레임워크에서 Ollama를 통해 실행되는 LLM(Local LLM)을 직접 제어하고 사용할 수 있게 해주는 통합 모듈

In [ ]:
%pip install langchain-ollama

LLM 객체 만들기

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="midm-2.0-base-instruct-q5_k_m",
    temperature=0.7, # 창의성 (0.0 ~ 1.0)
)

In [ ]:
from langchain_ollama import ChatOllama

# Ollama 모델 가져오기
llm = ChatOllama(model="midm-2.0-base-instruct-q5_k_m")

# 물어보기
resposne = llm.invoke("대한민국 육군에 대해 설명해줘")

print(resposne.content)

- JSON 형식으로 출력하기
    - `format` 파라미터를 이용하여 json 형태 지정 가능
    - `"reason in JSON format."` 이 프롬프트에 작성이 되어야함.

In [ ]:
from langchain_ollama import ChatOllama

# Ollama 모델 가져오기
llm = ChatOllama(
    model="midm-2.0-base-instruct-q5_k_m",
    format='json' # 출력 형식 지정
    )

# 물어보기
response = llm.invoke("대한민국 군대에 대해 설명해줘")
print(response.content)

`from_template()` 메소드로 `PromptTemplate` 만들기
- 변수로 지정하고 싶은 영역을 `{변수}`로 지정한다.

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
template = "다음의 키워드를 군대에 관련지어 응답을 해주세요. : {input}"

prompt = PromptTemplate.from_template(template)

print(prompt)

`{변수}`로 지정한 변수에 값을 넣어서 prompt를 완성 할 수 있다.

In [ ]:
prompt = prompt.format(input="음식")

print(prompt)

In [ ]:
print(llm.invoke(prompt));

`input_variables`를 사용하여 기본값을 지정할 수 있다.

In [ ]:
template = "{input1} 과 {input2}의 대표적인 부대는 각각 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["input1"],
    partial_variables={"input2": "공군"},
)

print(prompt)

In [ ]:
prompt.format(input1 = "육군")

`prompt.partial`을 통해 값을 재정의 할 수 있다.
- prompt.partial은 일부 변수의 값을 미리 지정해두고, 나머지 변수만 나중에 입력받고 싶을 때 사용한다.
- 예를 들어, 여러 번 반복적으로 사용할 값(예: input2="해군")을 미리 고정해두고, input1만 바꿔가며 사용할 때 유용하다.

In [ ]:
prompt = prompt.partial(input2="해군");

print(prompt)

input 값을 바꿔보자

In [ ]:
prompt_partial = prompt.format(input1="육군")

llm.invoke(prompt_partial)

## ChatPromptTemplate
- `ChatPromptTemplate`은 여러 역할의 메시지를 순서대로 넣어서 대화 프롬프트를 쉽게 만들 수 있는 도구
- 챗봇처럼 대화가 오갈 때 각 메시지를 따로 템플릿으로 만들고, 한 번에 조합해서 사용할 수 있음.
- 구조
   - 각 메시지는 역할(role)과 내용(content)로 구성됨
   - role의 종류
     - system: 시스템의 기본 지침이나 규칙을 안내할 때 사용
     - human: 사용자의 질문이나 요청을 입력할 때 사용
     - ai: AI의 답변을 입력할 때 사용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 군대 전문가 입니다. 당신은 항상 대답의 끝에 `충성!` 을 붙입니다."),
        ("human", "안녕하세요. 군대에 대해 궁금한 점이 있습니다."),
        ("ai", "안녕하세요. 무엇이든 질문해주세요 충성!"),
        ("human", "{input}"),
    ]
)

chat_prompt = chat_template.format_messages(input="너는 누구야?")

print(chat_prompt)

In [ ]:
llm.invoke(chat_prompt)

### MessagePlaceholder
 - `MessagePlaceholder`는 `ChatPromptTemplate`에서 메시지를 동적으로 삽입할 수 있게 해주는 기능
 - 기존 메시지를 재사용하거나 조건에 따라 다른 메시지를 넣을 때 유용함
 - 예를 들어 이전 대화 내용을 기억하고 참조해야 할 때 사용할 수 있음
 - 주요 사용 사례:
   - 대화 기록 유지
   - 동적 시스템 메시지 삽입
   - 조건부 메시지 포함


`HumanMessage, AIMessage, SystemMessage`
- 각각 사람, AI, 시스템의 메시지를 나타내는 객체
- ChatPromptTemplate에서 메시지를 더 명시적이고 타입 안전하게 다룰 수 있음
- 주요 특징:
  - SystemMessage: 시스템의 기본 설정이나 규칙을 정의할 때 사용
  - HumanMessage: 사용자의 입력을 나타낼 때 사용 
  - AIMessage: AI의 응답을 나타낼 때 사용
- 문자열 튜플 대신 이러한 객체를 사용하면 코드의 가독성과 유지보수성이 향상됨


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 군대 관련 챗봇 프롬프트 템플릿
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 군대 생활 도우미 챗봇입니다. 이전 대화를 참고하여 답변하세요."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{user_input}")
    ]
)

# 실제 군대 챗봇 대화 기록 (짧고 간단)
chat_history = [
    HumanMessage(content="안녕하세요"),
    AIMessage(content="안녕하세요! 군대 생활에 대해 궁금한 점이 있으면 언제든 물어보세요."),
    HumanMessage(content="다음 주에 입대예정이에요"),
    AIMessage(content="입대를 앞두고 계시는군요! 준비할 것들이 많으실 텐데 어떤 도움이 필요하신가요?")
]

# 이전 대화를 참고한 새로운 질문
messages = chat_prompt.format_messages(
    chat_history=chat_history,
    user_input="꼭 준비해야 할 물건이 뭐가 있나요?" # 사용자입력
)

print(messages)

In [ ]:

llm.invoke(messages)